In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
from my_package.textfile2df import nnlist2df


def bool_CO3_contained_poscar(POSCAR_nnlist):
    df_nnlist = nnlist2df(POSCAR_nnlist)
    
    # df_nnlistでcentral speciesがCのものに絞る
    df_nnlist_central_species_C = df_nnlist[df_nnlist['central species'] == 'C']
    
    # さらに，炭酸イオンかどうかを判定するためにに，あるcentral atomのneighboring speciesがCとO３つの計４つでできているか確認したい
    # central atomの値を入力すれば，neighboring speciesのリストを返す関数get_neighboring_species_list()を作成
    def get_neighboring_species_list(central_atom_id, df=df_nnlist_central_species_C):
            """
            To get all central atoms of a cluster(:neighbors), Input a number of cluster center element number(:central atom)

            Input: central atom column element In df_nnlist
         -> Output: All neighboring atom column element that Input(:elemnt) match central atom column element

            param1: Input: central atom column element In df_nnlist
            """
            # 左側の列から対応する行を選択し、右側の数値を取得
            # result = df_nnlist[df_nnlist['central atom'] == input_value]['neighboring atom'].values
            neighboring_species_list = df[df['central atom'] == central_atom_id]['neighboring species'].tolist()
            return neighboring_species_list
    

    def match_C_O_3(central_atom_id):
        # 原子団の要素数は，Cが1つ，Oが3つの計4つかどうかcheck
        if len(get_neighboring_species_list(central_atom_id)) == 4:
            # 原子団の要素にCが1つだけ含まれているかどうかcheck
            if get_neighboring_species_list(central_atom_id).count('C') == 1:
                # 原子団の要素にOがちょうど3つ含まれているかどうかcheck
                if get_neighboring_species_list(central_atom_id).count('O') == 3:
                    return True
            else:
                return False
        else:
            return False
        
        
    # df_nnlist_central_species_Cに対し，CO3がどうかを確認し，CO3である原子のid一覧を取得
    # まず，中心元素がCのid一覧(central atomの値の一覧)を取得
    central_species_C_list = df_nnlist_central_species_C['central atom'].unique()
    # その中で，match_C_O_3()を用いて，過不足なくCO3だけを含むものに絞る
    matched_central_species_C_list = [i for i in central_species_C_list if match_C_O_3(i)]
     
    return  True if len(matched_central_species_C_list) > 0 else False

In [4]:
import numpy as np
# C_O_existed_pos_nnlist_path_listを.npyからload
C_O_existed_pos_nnlist_path_list_loaded = np.load('../scripts_mk_C_O_contained_poscar_filter/C_O_existed_pos_nnlist_path_list.npy', allow_pickle=True)

In [5]:
C_O_existed_pos_nnlist_path_list_loaded

array(['../cif/1/00/00/1000000/POSCAR.nnlist',
       '../cif/1/00/00/1000001/POSCAR.nnlist',
       '../cif/1/00/00/1000002/POSCAR.nnlist', ...,
       '../cif/9/01/78/9017813/POSCAR.nnlist',
       '../cif/9/01/78/9017837/POSCAR.nnlist',
       '../cif/9/01/79/9017907/POSCAR.nnlist'], dtype='<U36')

In [6]:
C_O_existed_pos_nnlist_abspath_list = ['/mnt/ssd_elecom_black_c2c/' + i[3:] for i in C_O_existed_pos_nnlist_path_list_loaded]

In [84]:
# set([i[-7:] for i in C_O_existed_pos_nnlist_path_list_loaded])

In [ ]:
# check exist of C_O_existed_pos.nnlist
import os
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
p = Pool(cpu_count() - 1)
try:
    bool_exist_or_not = list(tqdm(p.imap(os.path.exists, C_O_existed_pos_nnlist_abspath_list), total=len(C_O_existed_pos_nnlist_abspath_list)))
finally:
    p.close()
    p.join()

In [7]:
set(bool_exist_or_not)

NameError: name 'bool_exist_or_not' is not defined

In [72]:
# # sample 
# set([bool_CO3_contained_poscar(i) for i in C_O_existed_pos_nnlist_abspath_list[0:1000]])

{True}

In [37]:
bool_CO3_contained_poscar(C_O_existed_pos_nnlist_abspath_list[12])

True

In [41]:
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
p = Pool(cpu_count() - 1)
try:
    bool_CO3_contained_nnlist_list = list(tqdm(p.imap(bool_CO3_contained_poscar, C_O_existed_pos_nnlist_abspath_list), total=len(C_O_existed_pos_nnlist_abspath_list)))
finally:
    p.close()
    p.join()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 205732/205732 [1:06:58<00:00, 51.20it/s]


In [57]:
len(np.array(C_O_existed_pos_nnlist_abspath_list)[bool_CO3_contained_nnlist_list])

1100

In [58]:
len(bool_CO3_contained_nnlist_list)

205732

In [55]:
CO3_contained_nnlist_abspath_list = np.array(C_O_existed_pos_nnlist_abspath_list)[bool_CO3_contained_nnlist_list]

In [56]:
np.save('CO3_contained_nnlist_abspath_list.npy' ,CO3_contained_nnlist_abspath_list )

In [61]:
CO3_contained_nnlist_abspath_list_loaded = np.load('CO3_contained_nnlist_abspath_list.npy', allow_pickle=True)

In [62]:
CO3_contained_nnlist_abspath_list_loaded

array(['/mnt/ssd_elecom_black_c2c/cif/1/00/00/1000033/POSCAR.nnlist',
       '/mnt/ssd_elecom_black_c2c/cif/1/00/00/1000052/POSCAR.nnlist',
       '/mnt/ssd_elecom_black_c2c/cif/1/00/03/1000320/POSCAR.nnlist', ...,
       '/mnt/ssd_elecom_black_c2c/cif/9/01/77/9017727/POSCAR.nnlist',
       '/mnt/ssd_elecom_black_c2c/cif/9/01/78/9017813/POSCAR.nnlist',
       '/mnt/ssd_elecom_black_c2c/cif/9/01/78/9017837/POSCAR.nnlist'],
      dtype='<U59')